In [1]:
import polars as pl

In [15]:
pl.read_csv("App/datasets/dataset11_lv_dnarna/runs/run_1/feat_extraction/Tsallis.csv", ignore_errors=True) \
.select(pl.all().exclude("nameseq"), pl.col("nameseq"))\
.filter(~pl.col("nameseq").str.contains("nameseq")) \
.set_sorted("nameseq")

k-tsallis12.3,k-tsallis22.3,k-tsallis32.3,k-tsallis42.3,k-tsallis52.3,label,nameseq
f64,f64,f64,f64,f64,str,str
0.600994,0.728334,0.755018,0.760864,0.762194,"""negative""","""-sample"""
0.628107,0.740318,0.756621,0.759818,0.761081,"""negative""","""-sample"""
0.61398,0.732599,0.754893,0.760864,0.762194,"""negative""","""-sample"""
0.610519,0.727303,0.751481,0.760341,0.762194,"""negative""","""-sample"""
0.623592,0.735025,0.756621,0.761387,0.762194,"""negative""","""-sample"""
…,…,…,…,…,…,…
0.633963,0.736455,0.756128,0.759818,0.761081,"""positive""","""+sample"""
0.633963,0.738939,0.758592,0.760864,0.761637,"""positive""","""+sample"""
0.579598,0.710172,0.74529,0.755017,0.759272,"""positive""","""+sample"""
